In [6]:
import sys
 
sys.path.append('..')
from piece_v2 import piece_v2

piece_v2.start(should_send_to_score=False)

In [7]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import *
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.modular.instrument import AddAction
from soundmining_tools.generative import *
from piece_v2 import *
from soundmining_tools.spectrum import make_fm_synthesis
import math
from enum import StrEnum
from soundmining_tools.modular_v2.synth_player_v2 import SynthNoteV2
from soundmining_tools.modular.instrument import NodeId
from soundmining_tools.sequencer import Sequencer, SequenceNote
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
import ipywidgets as widgets

piece_v2.reset()

if piece_v2.synth_player.should_send_to_score:
    score = piece_v2.synth_player.supercollider_score
    score.add_message(supercollider_client.group_head(0, instrument.NodeId.SOURCE.value))
    score.add_message(supercollider_client.group_tail(NodeId.SOURCE.value, NodeId.EFFECT.value))
    score.add_message(supercollider_client.group_tail(NodeId.EFFECT.value, NodeId.ROOM_EFFECT.value))
    score.add_message(supercollider_client.load_dir(V2_SYNTH_DIR))    

piece_v2.synth_player.start()

static_control = piece_v2.instruments.static_control
sine_control = piece_v2.instruments.sine_control
perc_control = piece_v2.instruments.perc_control
line_control = piece_v2.instruments.line_control
signal_sum = piece_v2.instruments.signal_sum
signal_multiply = piece_v2.instruments.signal_multiply
three_block_control = piece_v2.instruments.three_block_control
four_block_control = piece_v2.instruments.four_block_control

fundamental = 55
phi = (1 + math.sqrt(5)) / 2
inv_phi = 1 / phi

mod_ratio1 = phi
mod_ratio2 = inv_phi

fm_1 = make_fm_synthesis(fundamental, fundamental * mod_ratio1, 12)
fm_2 = make_fm_synthesis(fundamental, fundamental * mod_ratio2, 12)

negative_fm_2 = [n[1] for n in fm_2]
print(negative_fm_2)

# Low: 1, 2
# Middle low: 3, 0, 4
# Middle high: 5, 6, 7
# High: 8, 9, 10, 11

NoteType = StrEnum("NoteType", ["SINE", "SAW_PULSE"])

NoteStart = StrEnum("NoteStart", ["EARLY", "MIDDLE", "LATE"])

def sine_note(start: float, duration: float, amp: float, pan: tuple[float, float], pitch: float, mod_ratio1: float, mod_ratio2: float, effect: SynthNoteV2, track: str) -> SequenceNote: 
    mod_index1 = sine_control(2 * amp, 5 * amp)
    
    mod_freq1 = static_control(pitch * mod_ratio1)                
    mod_amp1 = signal_multiply(static_control(pitch * mod_ratio1), mod_index1).add_action(AddAction.TAIL_ACTION)
    mod1 = (
        piece_v2.synth_player.note()
        .sine(freq=mod_freq1, amp=mod_amp1)            
        .audio_stack.pop()
    )

    mod_index2 = sine_control(2 * amp, 5 * amp)
    mod_freq2 = static_control(pitch * mod_ratio2)                
    mod_amp2 = signal_multiply(static_control(pitch * mod_ratio2), mod_index2).add_action(AddAction.TAIL_ACTION)
    fm_mod1 = signal_sum(mod_freq2, mod1).add_action(AddAction.TAIL_ACTION)
    mod2 = (
        piece_v2.synth_player.note()
        .sine(freq=fm_mod1, amp=mod_amp2)            
        .audio_stack.pop()
    )

    car_ratio = 5/1
    #car_ratio = 1
    car_freq = static_control(pitch * car_ratio)
    car_amp = sine_control(0, amp)        
    fm_mod = signal_sum(car_freq, mod2).add_action(AddAction.TAIL_ACTION)
    pan_start, pan_end = pan
    if effect:
        (
            piece_v2.synth_player.note()                
                .sine(freq=fm_mod, amp=car_amp)                                
                .pan(line_control(pan_start, pan_end))                
                .send_to_synth_note(effect, start, duration)
        )
    else:
        (
            piece_v2.synth_player.note()                
                .sine(freq=fm_mod, amp=car_amp)
                .pan(line_control(pan_start, pan_end))
                .play(start, duration)                
        )
    return SequenceNote(start=start, track=track, duration=duration, freq=pitch)
    
def saw_pulse_note(start: float, duration: float, amp: float, pan: tuple[float, float], pitch: float, mod_ratio1: float, mod_ratio2: float, effect: SynthNoteV2, track: str): 
    mod_index1 = sine_control(2 * amp, 5 * amp)
    mod_freq1 = static_control(pitch * mod_ratio1)                
    mod_amp1 = signal_multiply(static_control(pitch * mod_ratio1), mod_index1).add_action(AddAction.TAIL_ACTION)
    mod1 = (
        piece_v2.synth_player.note()
        .saw(freq=mod_freq1, amp=mod_amp1)            
        .audio_stack.pop()
    )

    mod_index2 = sine_control(2 * amp, 5 * amp)
    
    mod_freq2 = static_control(pitch * mod_ratio2)                
    mod_amp2 = signal_multiply(static_control(pitch * mod_ratio2), mod_index2).add_action(AddAction.TAIL_ACTION)
    fm_mod1 = signal_sum(mod_freq2, mod1).add_action(AddAction.TAIL_ACTION)
    mod2 = (
        piece_v2.synth_player.note()            
        .pulse(freq=fm_mod1, width=line_control(random_range(0, 1), random_range(0, 1)), amp=mod_amp2)
        .audio_stack.pop()
    )

    car_ratio = 5/1
    car_freq = static_control(pitch * car_ratio)
    car_amp = sine_control(0, amp)        
    fm_mod = signal_sum(car_freq, mod2).add_action(AddAction.TAIL_ACTION)
    pan_start, pan_end = pan
    if effect:
        (
            piece_v2.synth_player.note()                
                .sine(freq=fm_mod, amp=car_amp)
                .pan(line_control(pan_start, pan_end))                
                .send_to_synth_note(effect, start, duration)
        )
    else:
        (
            piece_v2.synth_player.note()                
                .sine(freq=fm_mod, amp=car_amp)
                .pan(line_control(pan_start, pan_end))
                .play(start, duration)                
        )
    return SequenceNote(start=start, track=track, duration=duration, freq=pitch)

class LowSequence:
    SINE_EFFECT = 0
    SAW_EFFECT = 2
    SINE_TRACK = "Low sine"
    SAW_TRACK = "Low saw"

    def make_note_chain() -> MarkovChain:
        return MarkovChain({
            2: {2: 0.2, 1: 0.8},
            1: {2: 0.8, 1: 0.2}
        }, 1)

    note_chain = make_note_chain()
    short_note_chain = make_note_chain()
    saw_note_chain = make_note_chain()

    @classmethod
    def make_effect(cls, start_time: float, duration: float, output_bus: int) -> SynthNoteV2:
        reject_freq = 200
        bw = 600
        static_amp_factor = 1
        rq = bw / reject_freq
        amp_factor = (1 / math.sqrt(rq)) * static_amp_factor    
        effect = (
            piece_v2.synth_player.note(NodeId.EFFECT)
                .stereo_input()
                .stereo_band_reject_filter(static_control(reject_freq), static_control(rq))
                .stereo_volume(static_control(amp_factor)))
        (
            piece_v2.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)                
                .stereo_volume(static_control(1.0))                            
                .play(start_time, duration, output_bus=output_bus)
        )
        return effect

    short_note_start_chain = MarkovChain({
        NoteStart.LATE: {NoteStart.LATE: 0.8, NoteStart.EARLY: 0.2},
        NoteStart.EARLY: {NoteStart.LATE: 1, NoteStart.EARLY: 0}
    }, NoteStart.EARLY)
    
    @classmethod
    def play_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.note_chain.next()
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.2, 0.2)])
            duration = random_range(8, 13)
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))
            
            next_start = random.choice([NoteStart.EARLY, NoteStart.LATE])
            match next_start:
                case NoteStart.EARLY:
                    time += duration * random_range(0.15, 0.33)
                case NoteStart.LATE:             
                    time += duration * random_range(0.66, 0.85)
        return notes
    
    
    @classmethod
    def play_short_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.short_note_chain.next()
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.2, 0.2)])
            duration = random_range(3, 5)        
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))            

            next_start = cls.short_note_start_chain.next()
        
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(2, 3)
                case NoteStart.LATE:             
                    time += random_range(8, 13)
        return notes
    
    @classmethod
    def play_saw_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        saw_effect = cls.make_effect(start, duration + 13, cls.SAW_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.saw_note_chain.next()
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.2, 0.2)])
            duration = random_range(3, 5)        
            notes.append(saw_pulse_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=saw_effect, track=cls.SAW_TRACK))

            next_start = cls.short_note_start_chain.next()
        
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(1, 2)
                case NoteStart.LATE:             
                    time += random_range(13, 21)
        return notes


class MiddleLowSequence:
    SINE_EFFECT = 4
    SAW_EFFECT = 6
    SINE_TRACK = "Middle low sine"
    SAW_TRACK = "Middle low saw"

    def make_note_chain() -> MarkovChain:
        return MarkovChain({
            0: {3: 0.5, 4: 0.5},
            3: {0: 0.5, 4: 0.5},
            4: {0: 0.5, 3: 0.5},
        }, 0)
        

    note_chain = make_note_chain()    
    short_note_chain = make_note_chain()    
    saw_note_chain = make_note_chain()    

    @classmethod
    def make_effect(cls, start_time: float, duration: float, output_bus: int) -> SynthNoteV2:
        reject_freq = 400
        bw = 600
        static_amp_factor = 0.75
        rq = bw / reject_freq
        amp_factor = (1 / math.sqrt(rq)) * static_amp_factor    
        effect = (
            piece_v2.synth_player.note(NodeId.EFFECT)
                .stereo_input()
                .stereo_band_reject_filter(static_control(reject_freq), static_control(rq))
                .stereo_volume(static_control(amp_factor)))

        (
            piece_v2.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)                
                .stereo_volume(static_control(1.0))            
                .play(start_time, duration, output_bus=output_bus)
        )

        return effect
        
    @classmethod
    def play_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        

        time = start
        end = start + duration    

        while time < end:
            note = cls.note_chain.next()            
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.5, -0.2), (0.2, 0.5)])
            duration = random_range(8, 13)                    
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))            
            next_start = random.choice([NoteStart.EARLY, NoteStart.LATE])
            match next_start:
                case NoteStart.EARLY:
                    time += duration * random_range(0.15, 0.33)
                case NoteStart.LATE:             
                    time += duration * random_range(0.66, 0.85)
        return notes

    short_note_start_chain = MarkovChain({
        NoteStart.LATE: {NoteStart.LATE: 0.8, NoteStart.EARLY: 0.2},
        NoteStart.EARLY: {NoteStart.LATE: 1, NoteStart.EARLY: 0}
    }, NoteStart.EARLY)

    @classmethod
    def play_short_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.short_note_chain.next()            
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.5, -0.2), (0.2, 0.5)])
            duration = random_range(3, 5)                    
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))

            next_start = cls.short_note_start_chain.next()
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(2, 3)
                case NoteStart.LATE:             
                    time += random_range(8, 13)
        return notes
    
    @classmethod
    def play_saw_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []        
        saw_effect = cls.make_effect(start, duration + 13, cls.SAW_EFFECT)
        time = start
        end = start + duration    

        while time < end:
            note = cls.short_note_chain.next()            
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.5, -0.2), (0.2, 0.5)])
            duration = random_range(2, 3)                    
            notes.append(saw_pulse_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=saw_effect, track=cls.SAW_TRACK))

            next_start = cls.short_note_start_chain.next()
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(1, 2)
                case NoteStart.LATE:             
                    time += random_range(13, 21)
        return notes

class MiddleHighSequence:
    SINE_EFFECT = 8
    SAW_EFFECT = 10
    SINE_TRACK = "Middle high sine"
    SAW_TRACK = "Middle high saw"

    def make_note_chain() -> MarkovChain:
        return MarkovChain({
            5: {6: 0.5, 7: 0.5},
            6: {5: 0.5, 7: 0.5},
            7: {5: 0.5, 6: 0.5},
        }, 5)
            
    note_chain = make_note_chain()
    short_note_chain = make_note_chain()
    saw_note_chain = make_note_chain()

    @classmethod
    def make_effect(cls, start_time: float, duration: float, output_bus: int) -> SynthNoteV2:
        reject_freq = 600
        bw = 600
        static_amp_factor = 0.5
        rq = bw / reject_freq
        amp_factor = (1 / math.sqrt(rq)) * static_amp_factor    
        effect = (
            piece_v2.synth_player.note(NodeId.EFFECT)
                .stereo_input()
                .stereo_band_reject_filter(static_control(reject_freq), static_control(rq))
                .stereo_volume(static_control(amp_factor)))

        (
            piece_v2.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)                
                .stereo_volume(static_control(1.0))                            
                .play(start_time, duration, output_bus=output_bus)
        )
        return effect
        
    @classmethod
    def play_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.note_chain.next()                  
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.75, -0.5), (0.5, 0.75)])        
            duration = random_range(8, 13)        
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))            
            next_start = random.choice([NoteStart.EARLY, NoteStart.LATE])
            match next_start:
                case NoteStart.EARLY:
                    time += duration * random_range(0.15, 0.33)
                case NoteStart.LATE:             
                    time += duration * random_range(0.66, 0.85)
        return notes

    short_note_start_chain = MarkovChain({
        NoteStart.LATE: {NoteStart.LATE: 0.8, NoteStart.EARLY: 0.2},
        NoteStart.EARLY: {NoteStart.LATE: 1, NoteStart.EARLY: 0}
    }, NoteStart.EARLY)

    @classmethod
    def play_short_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.short_note_chain.next()            
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.75, -0.5), (0.5, 0.75)])        
            duration = random_range(3, 5)                    
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))            
            next_start = cls.short_note_start_chain.next()
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(2, 3)
                case NoteStart.LATE:             
                    time += random_range(8, 13)
        return notes
    

    @classmethod
    def play_saw_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []        
        saw_effect = cls.make_effect(start, duration + 13, cls.SAW_EFFECT)
        time = start
        end = start + duration    

        while time < end:
            note = cls.short_note_chain.next()                   
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.75, -0.5), (0.5, 0.75)])        
            duration = random_range(1, 2)                                
            notes.append(saw_pulse_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=saw_effect, track=cls.SAW_TRACK))
            next_start = cls.short_note_start_chain.next()
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(1, 2)
                case NoteStart.LATE:             
                    time += random_range(13, 21)
        return notes
    

class HighSequence:
    SINE_EFFECT = 12
    SAW_EFFECT = 14
    SINE_TRACK = "High sine"
    SAW_TRACK = "High saw"

    def make_note_chain() -> MarkovChain:
        return MarkovChain({
        8: {9: 0.33, 10: 0.33, 11: 0.33},
        9: {8: 0.33, 10: 0.33, 11: 0.33},
        10: {8: 0.33, 9: 0.33, 10: 0.33},
        11: {8: 0.33, 9: 0.33, 11: 0.33},
    }, 8)

    note_chain = make_note_chain()
    short_note_chain = make_note_chain()    
    saw_note_chain = make_note_chain()    

    short_note_start_chain = MarkovChain({
        NoteStart.LATE: {NoteStart.LATE: 0.8, NoteStart.EARLY: 0.2},
        NoteStart.EARLY: {NoteStart.LATE: 1, NoteStart.EARLY: 0}
    }, NoteStart.EARLY)

    @classmethod
    def make_effect(cls, start_time: float, duration: float, output_bus: int) -> SynthNoteV2:
        reject_freq = 1500
        bw = 600
        static_amp_factor = 0.25
        rq = bw / reject_freq
        amp_factor = (1 / math.sqrt(rq)) * static_amp_factor    
        effect = (
            piece_v2.synth_player.note(NodeId.EFFECT)
                .stereo_input()
                .stereo_band_reject_filter(static_control(reject_freq), static_control(rq))
                .stereo_volume(static_control(amp_factor)))

        (
            piece_v2.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)                
                .stereo_volume(static_control(1.0))           
                .play(start_time, duration, output_bus=output_bus)
        )
        return effect
        
    @classmethod
    def play_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.note_chain.next()             
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-1, -0.75), (0.75, 1)])
            duration = random_range(8, 13)                    
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))
            next_start = random.choice([NoteStart.EARLY, NoteStart.LATE])
            match next_start:
                case NoteStart.EARLY:
                    time += duration * random_range(0.15, 0.33)
                case NoteStart.LATE:             
                    time += duration * random_range(0.66, 0.85)
        return notes

    @classmethod
    def play_short_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []
        sine_effect = cls.make_effect(start, duration + 13, cls.SINE_EFFECT)        
        time = start
        end = start + duration    

        while time < end:
            note = cls.short_note_chain.next()            
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-1, -0.75), (0.75, 1)])
            duration = random_range(3, 5)        
            notes.append(sine_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=sine_effect, track=cls.SINE_TRACK))
            next_start = cls.short_note_start_chain.next()
            
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(2, 3)
                case NoteStart.LATE:             
                    time += random_range(8, 13)
        return notes
    
    @classmethod
    def play_saw_sequence(cls, start: float, duration: float) -> list[SequenceNote]:
        notes = []        
        saw_effect = cls.make_effect(start, duration + 13, cls.SAW_EFFECT)
        time = start
        end = start + duration    

        while time < end:
            note = cls.short_note_chain.next()                 
            pitch = negative_fm_2[note]
            amp = random_range(0.15, 0.85)
            pan = pan_line(random_range(0.2, 0.5), ranges=[(-1, -0.75), (0.75, 1)])
            duration = random_range(2, 3)        
            notes.append(saw_pulse_note(start=time, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=saw_effect, track=cls.SAW_TRACK))
            next_start = cls.short_note_start_chain.next()
            
            match next_start:
                case NoteStart.EARLY:
                    time += random_range(1, 2)
                case NoteStart.LATE:             
                    time += random_range(13, 21)
        return notes

class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)    
    
    def handle_note(self, patch_arguments: PatchArguments) -> None:
        pitch = negative_fm_2[patch_arguments.note]
        amp = patch_arguments.amp
        start = patch_arguments.start
        pan = pan_line(random_range(0.2, 0.5), ranges=[(-0.5, -0.2), (0.2, 0.5)])
        duration = random_range(13, 21)

        match patch_arguments.octave:
            case 2:
                saw_pulse_note(start=start, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=None, track="Saw Note")                    
            case 3: 
                sine_note(start=start, duration=duration, amp=amp, pan=pan, pitch=pitch, mod_ratio1=mod_ratio1, mod_ratio2=mod_ratio2, effect=None, track="Sine Note")  
            case 4:
                match patch_arguments.note:
                    case 0:
                        LowSequence.play_sequence(patch_arguments.start, 55)                
                    case 1:
                        LowSequence.play_short_sequence(patch_arguments.start, 55)
                    case 2:
                        LowSequence.play_saw_sequence(patch_arguments.start, 55)                        
                    case 3: 
                        MiddleLowSequence.play_sequence(patch_arguments.start, 55)
                    case 4: 
                        MiddleLowSequence.play_short_sequence(patch_arguments.start, 55)
                    case 5: 
                        MiddleLowSequence.play_saw_sequence(patch_arguments.start, 55)    
                    case 6: 
                        MiddleHighSequence.play_sequence(patch_arguments.start, 55)
                    case 7: 
                        MiddleHighSequence.play_short_sequence(patch_arguments.start, 55)
                    case 8: 
                        MiddleHighSequence.play_saw_sequence(patch_arguments.start, 55)    
                    case 9: 
                        HighSequence.play_sequence(patch_arguments.start, 55)
                    case 10:
                        HighSequence.play_short_sequence(patch_arguments.start, 55)
                    case 11:
                        HighSequence.play_saw_sequence(patch_arguments.start, 55)


def make_piece(start: float) -> list[SequenceNote]:
    notes = []
    start_1 = start
    notes.extend(LowSequence.play_sequence(start_1, 55))
    notes.extend(MiddleHighSequence.play_sequence(start_1 + random_range(5, 13), 55))
    notes.extend(MiddleLowSequence.play_saw_sequence(start_1 + random_range(13, 21), 55))

    start_2 = start_1 + random_range(34, 55)
    notes.extend(HighSequence.play_sequence(start_2, 55))
    notes.extend(MiddleLowSequence.play_sequence(start_2 + random_range(5, 13), 55))
    notes.extend(LowSequence.play_saw_sequence(start_2 + random_range(13, 21), 55))

    start_3 = start_2 + random_range(34, 55)
    notes.extend(MiddleHighSequence.play_sequence(start_3, 55))
    notes.extend(MiddleLowSequence.play_sequence(start_3 + random_range(5, 13), 55))
    notes.extend(HighSequence.play_short_sequence(start_3 + random_range(13, 21), 55))

    start_4 = start_3 + random_range(34, 55)
    notes.extend(LowSequence.play_sequence(start_4, 55))
    notes.extend(HighSequence.play_sequence(start_4 + random_range(5, 13), 55))
    notes.extend(MiddleLowSequence.play_short_sequence(start_4 + random_range(13, 21), 55))

    start_5 = start_4 + random_range(34, 55)
    notes.extend(MiddleLowSequence.play_short_sequence(start_5, 55))
    notes.extend(MiddleHighSequence.play_short_sequence(start_5 + random_range(5, 13), 55))
    notes.extend(HighSequence.play_saw_sequence(start_5 + random_range(13, 21), 55))

    start_6 = start_5 + random_range(34, 55)
    notes.extend(HighSequence.play_short_sequence(start_6, 55))
    notes.extend(LowSequence.play_short_sequence(start_6 + random_range(5, 13), 55))
    notes.extend(MiddleLowSequence.play_saw_sequence(start_6 + random_range(13, 21), 55))

    start_7 = start_6 + random_range(34, 55)
    notes.extend(MiddleHighSequence.play_sequence(start_7, 55))
    notes.extend(MiddleLowSequence.play_sequence(start_7 + random_range(5, 13), 55))
    notes.extend(LowSequence.play_saw_sequence(start_7 + random_range(13, 21), 55))

    start_8 = start_7 + random_range(34, 55)
    notes.extend(LowSequence.play_sequence(start_8, 55))
    notes.extend(HighSequence.play_sequence(start_8 + random_range(5, 13), 55))
    notes.extend(MiddleLowSequence.play_saw_sequence(start_8 + random_range(13, 21), 55))

    return notes
        
my_handler = MyHandler(piece_v2.supercollider_client)
piece_v2.receiver.set_note_handler(my_handler)

notes = make_piece(0)
if piece_v2.synth_player.should_send_to_score:
    piece_v2.synth_player.supercollider_score.make_score_file("module-music-10-v1.txt")

ui_piece = UiPieceBuilder().add_notes(notes).build()

piece_duration = ui_piece.get_duration()

piece_stats = {"total": piece_duration, "total minutes": piece_duration / 60.0, "tracks": len(ui_piece.tracks)}

min_freq = 0
max_freq = 0

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
        min_freq = min(min_freq, note.freq)
        max_freq = max(max_freq, note.freq)

    piece_stats[track.track_name] = track_duration

display(piece_stats)

TRACK_HEIGHT = 100
NOTE_SCALE_FACTOR = 3
HEIGHT_INDENT = 80

ui_width = 200 + (piece_duration * NOTE_SCALE_FACTOR)
ui_height = TRACK_HEIGHT * len(ui_piece.tracks)

canvas = Canvas(width=ui_width, height=ui_height)

out = Output()

@out.capture()
def handle_mouse_down(x, y):
    canvas.flush()
    print("Mouse down event:", x, y)


canvas.on_mouse_down(handle_mouse_down)
canvas.global_alpha = 0.7

display(canvas)


with hold_canvas():

    canvas.clear()
    for track_index, track in enumerate(sorted(ui_piece.tracks, key=lambda tr: tr.track_name)):
        canvas.font = "14px sans-serif"
        canvas.fill_style = "Black"
        canvas.fill_text(
            track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT
        )
        canvas.stroke_style = "Black"
        canvas.stroke_lines(
            [
                (150, (track_index * TRACK_HEIGHT) + 10),
                (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10)),
            ]
        )
        for note in track.notes:
            relative_note = (note.freq - min_freq) / (max_freq - min_freq)
            startx = 200 + (note.start * NOTE_SCALE_FACTOR)
            starty = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR
            peaky = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
                - 5
            )
            endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
            endy = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            canvas.stroke_style = note.color
            canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])

stop_button = widgets.Button(description="Stop")
status = widgets.Output()
display(stop_button, status)
with status:
    print("Playing")

def stop_playback(b):
    piece_v2.reset()
    canvas.clear()
    status.clear_output()
    with status:
        print("Playback stopped")


stop_button.on_click(stop_playback)


[55.0, 21.008130618755786, 12.983738762488429, 46.97560814373264, 80.96747752497686, 114.95934690622107, 148.9512162874653, 182.9430856687095, 216.93495504995371, 250.92682443119793, 284.91869381244214, 318.91056319368636]


{'total': 370.84898022830987,
 'total minutes': 6.180816337138498,
 'tracks': 7,
 'Low sine': 358.558544522542,
 'Middle high sine': 310.11155392327186,
 'Middle low saw': 370.84898022830987,
 'High sine': 367.8301928896063,
 'Middle low sine': 313.74228689705416,
 'Low saw': 319.9733271479275,
 'High saw': 234.39176651409537}

Canvas(height=700, width=1312)

Button(description='Stop', style=ButtonStyle())

Output()

In [8]:
piece_v2.stop()